In [ ]:
!pip install taaled pylats spacy
# English models
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_trf

# Spanish models (used as fallback)
!python -m spacy download es_core_news_sm
!python -m spacy download es_dep_news_trf

!pip install transformers torch nltk

!pip install textblob
!python -m textblob.download_corpora

!pip install convokit

In [ ]:
import re
import os
import csv
import json
import time
from datetime import datetime
from tqdm import tqdm
from collections import Counter
import pandas as pd

import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
import textstat
from taaled import ld
from pylats import lats
from textblob import TextBlob

import torch
import torch.nn.functional as F
from transformers import pipeline, AutoModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from simpletransformers.ner import NERModel, NERArgs

from ollama import chat

In [ ]:
# Load the dataset
with open('../data/raw/semantic-web-journal.json', 'r') as f:
    data = json.load(f)

data[0]

In [1]:
output_file = "../data/processed/semantic-web-journal-analysis.csv"
input_file = output_file

In [ ]:
rows = []

for paper in data:
    paper_id = paper.get("id", "").strip()
    if paper_id.upper() == "UNK" or not paper_id:
        continue

    paper_date_str = paper.get("date", "")
    try:
        paper_date = datetime.strptime(paper_date_str, "%m/%d/%Y")
    except Exception:
        paper_date = None

    for review in paper.get("reviews", []):
        reviewer = review.get("reviewer", "Anonymous").strip()
        review_date_str = review.get("date", "").strip()

        # Clean review text
        review_text = review.get("comment", "")
        review_text = review_text.replace("\n", " ").replace("\r", " ").strip()
        review_suggestion = review.get("suggestion", "")

        length_words = len(review_text.split())

        try:
            review_date = datetime.strptime(review_date_str, "%d/%b/%Y")
            days_to_submit = (review_date - paper_date).days if paper_date else None
        except Exception:
            days_to_submit = None

        rows.append({
            "paper_id": paper_id,
            "reviewer": reviewer,
            "review_date": review_date_str,
            "review_suggestion": review_suggestion,
            "length_words": length_words,
            "days_to_submit": days_to_submit,
            "review_text": review_text
        })

# Save to CSV with proper quoting
with open(output_file, mode='w', newline='', encoding='utf-8', errors='ignore') as f:
    writer = csv.DictWriter(f, fieldnames=rows[0].keys(), quoting=csv.QUOTE_ALL)
    writer.writeheader()
    writer.writerows(rows)

print(f"✅ Cleaned and saved {len(rows)} reviews with full text to review_analysis.csv")



In [ ]:
output_rows = []

# Read rows
with open(input_file, mode='r', encoding='utf-8', errors='ignore') as f:
    reader = list(csv.DictReader(f))
    fieldnames = list(reader[0].keys())

    # Ensure 'mattr' column exists
    if "mattr" not in fieldnames:
        fieldnames.append("mattr")
    # Drop 'mattr_reason' if it exists
    if "mattr_reason" in fieldnames:
        fieldnames.remove("mattr_reason")

    for row in tqdm(reader, desc="Computing MATTR"):
        review_text = row.get("review_text", "").strip()
        mattr_value = ""

        try:
            cleaned = lats.Normalize(review_text, lats.ld_params_en)
            tokens = cleaned.toks
            mattr_value = f"{ld.lexdiv(tokens).mattr:.4f}"
        except Exception as e:
            mattr_value = ""

        row["mattr"] = mattr_value
        # Remove 'mattr_reason' if it exists in the row
        row.pop("mattr_reason", None)
        output_rows.append(row)

# Write updated file
with open(input_file, mode='w', newline='', encoding='utf-8', errors='ignore') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
    writer.writeheader()
    writer.writerows(output_rows)

print("✅ Clean MATTR values saved to review_analysis.csv")


In [ ]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("shahrukhx01/bert-mini-finetune-question-detection")
model = AutoModelForSequenceClassification.from_pretrained("shahrukhx01/bert-mini-finetune-question-detection")
model.eval()

output_rows = []

# Load review rows
with open(input_file, mode='r', encoding='utf-8', errors='ignore') as f:
    reader = list(csv.DictReader(f))
    fieldnames = list(reader[0].keys())
    if "question_count" not in fieldnames:
        fieldnames.append("question_count")

    for row in tqdm(reader, desc="Detecting Questions"):
        review_text = row.get("review_text", "")
        question_count = 0

        try:
            sentences = sent_tokenize(review_text)
            for sent in sentences:
                inputs = tokenizer(
                    sent,
                    return_tensors="pt",
                    truncation=True,
                    max_length=64,
                    padding=True
                )
                with torch.no_grad():
                    outputs = model(**inputs)
                    predicted = torch.argmax(outputs.logits, dim=1).item()

                    # Label 0 = question
                    if predicted == 0:
                        question_count += 1
        except Exception as e:
            question_count = ""

        row["question_count"] = question_count
        output_rows.append(row)

# Save updated CSV
with open(input_file, mode='w', newline='', encoding='utf-8', errors='ignore') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
    writer.writeheader()
    writer.writerows(output_rows)

print("✅ Questions counted and saved in review_analysis.csv")


In [ ]:
# --- Citation counting logic ---
def count_citations(text):
    citation_patterns = [
        r'\[\d+(?:,\s*\d+)*\]',                         # [1], [1, 2, 3]
        r'\([A-Za-z]+ et al\.,\s*\d{4}\)',               # (Smith et al., 2020)
        r'\(\d{4}[a-z]?\)',                              # (2020), (2020a)
        r'\[[A-Za-z]+\d{4}[a-z]?\]',                     # [Smith2020], [Johnson2021a]
        r'\b(?:doi:|arxiv:|https?://[^\s]+)',             # DOI, arXiv, URLs
    ]
    pattern = '|'.join(citation_patterns)
    matches = re.findall(pattern, text)
    return len(matches)

# --- Load CSV and apply ---
output_rows = []

with open(input_file, mode='r', encoding='utf-8', errors='ignore') as f:
    reader = list(csv.DictReader(f))
    fieldnames = list(reader[0].keys())

    # Update for citation_count
    if "citation_count" not in fieldnames:
        fieldnames.append("citation_count")
    if "has_citation" in fieldnames:
        fieldnames.remove("has_citation")  # Remove old 'has_citation' if needed

    for row in tqdm(reader, desc="Counting Citations"):
        review_text = row.get("review_text", "")
        citation_count = count_citations(review_text)
        row["citation_count"] = citation_count
        output_rows.append(row)

# --- Save updated CSV ---
with open(input_file, mode='w', newline='', encoding='utf-8', errors='ignore') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
    writer.writeheader()
    writer.writerows(output_rows)

print("✅ Citation counts added to review_analysis.csv")


In [ ]:
with open(output_file, mode='r', encoding='utf-8', errors='ignore') as f:
    reader = csv.DictReader(f)
    total = 0
    with_citations = 0

    for row in reader:
        total += 1
        if row.get("citation_count") == "2":
            with_citations += 1

print(f"📄 Total reviews: {total}")
print(f"🔍 Reviews with citations: {with_citations}")
print(f"📊 Percentage: {(with_citations / total * 100):.2f}%")

In [ ]:
output_rows = []

# Read and process the file
with open(input_file, mode='r', encoding='utf-8', errors='ignore') as f:
    reader = list(csv.DictReader(f))
    fieldnames = list(reader[0].keys())

    # Add new column if not already there
    if "sentiment_polarity" not in fieldnames:
        fieldnames.append("sentiment_polarity")

    for row in tqdm(reader, desc="Analyzing Sentiment"):
        review_text = row.get("review_text", "").strip()
        try:
            blob = TextBlob(review_text)
            sentiment = blob.sentiment.polarity
        except Exception:
            sentiment = ""

        row["sentiment_polarity"] = sentiment
        output_rows.append(row)

# Write updated CSV
with open(input_file, mode='w', newline='', encoding='utf-8', errors='ignore') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
    writer.writeheader()
    writer.writerows(output_rows)

print("✅ Sentiment polarity added to review_analysis.csv")

In [ ]:
# Load politeness classifier and tokenizer
classifier = pipeline("text-classification", model="Genius1237/xlm-roberta-large-tydip")
tokenizer = AutoTokenizer.from_pretrained("Genius1237/xlm-roberta-large-tydip")

# Load CSV
df = pd.read_csv(input_file)

# Helper function: chunk a long text
def chunk_text(text, tokenizer, chunk_size=500, overlap=100):
    if not isinstance(text, str):
        return []

    tokens = tokenizer.tokenize(text)
    chunks = []

    for start in range(0, len(tokens), chunk_size - overlap):
        end = start + chunk_size
        chunk_tokens = tokens[start:end]
        chunk_text = tokenizer.convert_tokens_to_string(chunk_tokens)
        chunks.append(chunk_text)

        if end >= len(tokens):
            break  # Stop if we're at the end of the token list

    return chunks

# Process each row and calculate politeness score (weighted by chunk length)
scores = []

for text in tqdm(df['review_text'], desc="Scoring politeness"):
    chunked_texts = chunk_text(text, tokenizer)
    weighted_sum = 0
    total_tokens = 0

    for chunk in chunked_texts:
        try:
            chunk_tokens = tokenizer.tokenize(chunk)
            token_count = len(chunk_tokens)

            result = classifier(chunk)[0]
            score = result['score'] if result['label'].lower() == 'polite' else 1 - result['score']

            weighted_sum += score * token_count
            total_tokens += token_count
        except Exception:
            continue  # skip on error

    if total_tokens > 0:
        weighted_avg_score = weighted_sum / total_tokens
    else:
        weighted_avg_score = None

    scores.append(weighted_avg_score)

# Add to DataFrame and save
df['politeness'] = scores
df.to_csv(input_file, index=False)
print("✅ Saved updated file with chunk-averaged 'politeness' scores.")


In [ ]:
# --- Load SPECTER model ---
model_name = "allenai/specter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

output_rows = []

with open(input_file, mode='r', encoding='utf-8', errors='ignore') as f:
    reader = list(csv.DictReader(f))
    fieldnames = list(reader[0].keys())

    if "similarity_score" not in fieldnames:
        fieldnames.append("similarity_score")

    for row in tqdm(reader, desc="Computing Relevance Score"):
        review_text = row.get("review_text", "")
        paper_id = row.get("paper_id", "").strip()

        try:
            # Find matching entry in data
            matched_entry = next((entry for entry in data if str(entry.get("id", "")).strip() == paper_id), None)

            if matched_entry:
                title = matched_entry.get("title", "")
                abstract = matched_entry.get("abstract", "")
                doc_text = f"{title} {abstract}"

                # Encode document
                doc_inputs = tokenizer(doc_text, return_tensors="pt", truncation=True, padding=True, max_length=512)
                doc_inputs = {k: v.to(device) for k, v in doc_inputs.items()}
                with torch.no_grad():
                    doc_emb = model(**doc_inputs).last_hidden_state[:, 0, :]  # [CLS]

                # Encode review text
                review_inputs = tokenizer(review_text, return_tensors="pt", truncation=True, padding=True, max_length=512)
                review_inputs = {k: v.to(device) for k, v in review_inputs.items()}
                with torch.no_grad():
                    review_emb = model(**review_inputs).last_hidden_state[:, 0, :]  # [CLS]

                # Cosine similarity
                similarity_score = F.cosine_similarity(doc_emb, review_emb).item()
                row["similarity_score"] = similarity_score

            else:
                row["similarity_score"] = ""

        except Exception as e:
            row["similarity_score"] = ""

        output_rows.append(row)

# --- Save updated CSV ---
with open(input_file, mode='w', newline='', encoding='utf-8', errors='ignore') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
    writer.writeheader()
    writer.writerows(output_rows)

print("✅ Relevance scores added to review_analysis.csv")


In [ ]:
output_rows = []

with open(input_file, mode='r', encoding='utf-8', errors='ignore') as f:
    reader = list(csv.DictReader(f))
    original_fieldnames = list(reader[0].keys())

    # Insert title and abstract at positions 5 and 6
    new_fieldnames = original_fieldnames[:5] + ["title", "abstract"] + original_fieldnames[5:]

    for row in tqdm(reader, desc="Adding Title and Abstract (Escaping Newlines)"):
        paper_id = row.get("paper_id", "").strip()

        # Find matching entry
        matched_entry = next((entry for entry in data if str(entry.get("id", "")).strip() == paper_id), None)

        if matched_entry:
            title = matched_entry.get("title", "")
            abstract = matched_entry.get("abstract", "")
        else:
            title = ""
            abstract = ""

        # Escape real newlines in title and abstract
        title = title.replace("\r\n", "\\n").replace("\n", "\\n")
        abstract = abstract.replace("\r\n", "\\n").replace("\n", "\\n")

        # Build new row
        new_row = {}
        for idx, field in enumerate(new_fieldnames):
            if field == "title":
                new_row[field] = title
            elif field == "abstract":
                new_row[field] = abstract
            else:
                # Map original fields
                original_field_idx = idx if idx < 5 else idx - 2  # Adjust because we inserted 2 fields
                if original_field_idx < len(original_fieldnames):
                    original_field = original_fieldnames[original_field_idx]
                    new_row[field] = row.get(original_field, "")

        output_rows.append(new_row)

# --- Save updated CSV ---
with open(input_file, mode='w', newline='', encoding='utf-8', errors='ignore') as f:
    writer = csv.DictWriter(f, fieldnames=new_fieldnames, quoting=csv.QUOTE_ALL)
    writer.writeheader()
    writer.writerows(output_rows)

print("✅ Title and Abstract (with clean \\n) added to review_analysis.csv")


In [ ]:
# Helper: parse dates consistently
def parse_date(date_str):
    try:
        return datetime.strptime(date_str, "%d/%b/%Y")
    except Exception:
        return None

output_rows = []

with open(input_file, mode='r', encoding='utf-8', errors='ignore') as f:
    reader = list(csv.DictReader(f))
    rows = list(reader)
    fieldnames = list(rows[0].keys())

    if "num_days_before_deadline" not in fieldnames:
        fieldnames.append("num_days_before_deadline")

    # First: find latest review_date per paper_id
    latest_review_dates = {}

    for row in rows:
        paper_id = row["paper_id"]
        review_date = parse_date(row["review_date"])

        if paper_id and review_date:
            if paper_id not in latest_review_dates:
                latest_review_dates[paper_id] = review_date
            else:
                if review_date > latest_review_dates[paper_id]:
                    latest_review_dates[paper_id] = review_date

    # Second: compute days before deadline for each review
    for row in tqdm(rows, desc="Computing num_days_before_deadline"):
        paper_id = row["paper_id"]
        review_date = parse_date(row["review_date"])
        deadline_date = latest_review_dates.get(paper_id)

        if review_date and deadline_date:
            days_before_deadline = (deadline_date - review_date).days
            row["num_days_before_deadline"] = days_before_deadline
        else:
            row["num_days_before_deadline"] = ""

        output_rows.append(row)

# --- Save updated CSV ---
with open(input_file, mode='w', newline='', encoding='utf-8', errors='ignore') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
    writer.writeheader()
    writer.writerows(output_rows)

print("✅ num_days_before_deadline added to review_analysis.csv")


In [ ]:
# Load CSV
df = pd.read_csv(input_file)

# Enable tqdm for pandas apply
tqdm.pandas(desc="Scoring Readability")

# Define the readability scoring function
def readability_scores(text):
    try:
        return {
            "flesch_reading_ease": textstat.flesch_reading_ease(text),
            "flesch_kincaid_grade": textstat.flesch_kincaid_grade(text),
            "gunning_fog": textstat.gunning_fog(text),
            "smog_index": textstat.smog_index(text),
            "automated_readability_index": textstat.automated_readability_index(text),
        }
    except:
        return {
            "flesch_reading_ease": None,
            "flesch_kincaid_grade": None,
            "gunning_fog": None,
            "smog_index": None,
            "automated_readability_index": None,
        }

# Apply function with progress bar
readability_results = df["review_text"].progress_apply(readability_scores)
readability_df = pd.DataFrame(readability_results.tolist())

# Merge new columns
df = pd.concat([df, readability_df], axis=1)

# Save to file
df.to_csv(output_file, index=False)
print(f"Saved enriched file to: {output_file}")


In [ ]:
# Define labels used by the HEDGEhog model
labels = ["C", "D", "E", "I", "N"]

# Set up model arguments
model_args = NERArgs()
model_args.labels_list = labels
model_args.silent = True
model_args.use_multiprocessing = False

# Initialize model
model = NERModel(
    model_type="bert",
    model_name="jeniakim/hedgehog",
    args=model_args,
    use_cuda=torch.cuda.is_available()
)

# Load the CSV
df = pd.read_csv(input_file)

# Function to count each label type
def count_hedge_labels(text):
    predictions, _ = model.predict([text])
    token_labels = [list(token.values())[0] for token in predictions[0]]
    counts = Counter(token_labels)
    return {label: counts.get(label, 0) for label in labels}

# Apply across review_text
tqdm.pandas(desc="Counting Hedge Labels")
hedge_counts = df["review_text"].progress_apply(count_hedge_labels)

# Convert counts into separate columns and join with df
hedge_df = pd.DataFrame(hedge_counts.tolist())
hedge_df.columns = [f"hedge_{label}" for label in hedge_df.columns]

df = pd.concat([df.reset_index(drop=True), hedge_df.reset_index(drop=True)], axis=1)

# Save updated CSV
df.to_csv(output_file, index=False)
print("✅ Hedge label counts saved to:", output_file)


In [ ]:
def count_explicit_references(review_text):
    """
    Counts the number of explicit references to parts of a paper in a review text.
    Keywords include: section, page, figure, paragraph, p., fig., and equation.
    
    Parameters:
        review_text (str): The review text to analyze.
    
    Returns:
        int: The number of keyword matches.
        list: The matched keywords (for inspection).
    """
    # Define reference keywords
    keywords = [
        r'\bsection\b',
        r'\bpage\b',
        r'\bfigure\b',
        r'\bparagraph\b',
        r'\bp\.\b',
        r'\bfig\.\b',
        r'\bequation\b'
    ]
    
    # Combine all keywords into a single regex pattern (case-insensitive)
    pattern = re.compile('|'.join(keywords), flags=re.IGNORECASE)

    # Find all matches
    matches = pattern.findall(review_text)

    return len(matches), matches

# # Example usage
# review = """
# As discussed in Section 3.2, the proposed method relies heavily on assumptions. 
# The results in Figure 4 support the claim, but the explanation on page 5 is not clear. 
# Please revise paragraph two on p. 6. Equation 2 should be elaborated further.
# """

# count, matched_keywords = count_explicit_references(review)
# print(f"Number of explicit references: {count}")
# print("Matched keywords:", matched_keywords)

# Load the CSV file
df = pd.read_csv(input_file)

# Add the new column
tqdm.pandas(desc="Counting explicit references")
df["explicit_references"] = df["review_text"].progress_apply(
    lambda text: count_explicit_references(text)[0] if pd.notna(text) else 0
)

# Save the updated CSV
df.to_csv(input_file, index=False, quoting=csv.QUOTE_ALL)

print("✅ Explicit reference counts added to CSV.")

In [2]:
# Load data
df = pd.read_csv(input_file)
llm_fields = [
    "llm_comprehensiveness", "llm_technical_terms", "llm_factuality",
    "llm_sentiment_polarity", "llm_politeness", "llm_vagueness",
    "llm_objectivity", "llm_fairness", "llm_actionability",
    "llm_constructiveness", "llm_relevance_alignment",
    "llm_clarity_readability", "llm_overall_score_100"
]

# Check for missing fields and add them if not present
for field in llm_fields:
    if field not in df.columns:
        df[field] = pd.NA

# Pattern to extract JSON block
pattern = re.compile(r"<review_assessment>\s*(\{.*?\})\s*</review_assessment>", re.DOTALL)

# Updated prompt template
template = """# REVIEW-QUALITY JUDGE

## 0 — ROLE

You are **ReviewInspector-LLM**, a rigorous, impartial meta-reviewer.
Your goal is to assess the quality of a single peer-review against a predefined set of criteria and to provide precise, structured evaluations.

## 1 — INPUTS

Title: {title}
Abstract: {abstract}
Review: {review_text}

## 2 — EVALUATION CRITERIA

Return **only** the scale value or label at right (no rationale text).

| #  | Criterion                    | Allowed scale / label                       | Description                                                                |
| -- | ---------------------------- | ------------------------------------------- | -------------------------------------------------------------------------- |
| 1  | **Comprehensiveness**        | integer **0-5**                             | Extent to which the review covers all key aspects of the paper.            |
| 2  | **Usage of Technical Terms** | integer **0-5**                             | Appropriateness and frequency of domain-specific vocabulary.               |
| 3  | **Factuality**               | **factual / partially factual / unfactual** | Accuracy of the statements made in the review.                             |
| 4  | **Sentiment Polarity**       | **negative / neutral / positive**           | Overall sentiment conveyed by the reviewer.                                |
| 5  | **Politeness**               | **polite / neutral / impolite**             | Tone and manner of the review language.                                    |
| 6  | **Vagueness**                | **none / low / moderate / high / extreme**  | Degree of ambiguity or lack of specificity in the review.                  |
| 7  | **Objectivity**              | integer **0-5**                             | Presence of unbiased, evidence-based commentary.                           |
| 8  | **Fairness**                 | integer **0-5**                             | Perceived impartiality and balance in judgments.                           |
| 9  | **Actionability**            | integer **0-5**                             | Helpfulness of the review in suggesting clear next steps.                  |
| 10 | **Constructiveness**         | integer **0-5**                             | Degree to which the review offers improvements rather than just criticism. |
| 11 | **Relevance Alignment**      | integer **0-5**                             | How well the review relates to the content and scope of the paper.         |
| 12 | **Clarity and Readability**  | integer **0-5**                             | Ease of understanding the review, including grammar and structure.         |
| 13 | **Overall Quality**          | integer **0-100**                           | Holistic evaluation of the review's usefulness and professionalism.        |

## 3 — SCORING GUIDELINES

For 0-5 scales:

* 5 = Outstanding
* 4 = Strong
* 3 = Adequate
* 2 = Weak
* 1 = Very weak
* 0 = Absent/irrelevant

## 4 — ANALYSIS & COMPUTATION (silent)

1. Read and understand the review in the context of the paper title and abstract.
2. Extract quantitative and qualitative signals (e.g., term usage, factual consistency, tone, clarity).
3. Map observations to the corresponding scoring scales.

## 5 — OUTPUT FORMAT (strict)  
Return **exactly one** JSON block wrapped in the tag below — **no comments or extra text**.

```json
<review_assessment>
{{
  "paper_title": "{title}",
  "criteria": {{
    "comprehensiveness":       ...,
    "technical_terms":         ...,
    "factuality":              ...,
    "sentiment_polarity":      ...,
    "politeness":              ...,
    "vagueness":               ...,
    "objectivity":             ...,
    "fairness":                ...,
    "actionability":           ...,
    "constructiveness":        ...,
    "relevance_alignment":     ...,
    "clarity_readability":     ...,
    "overall_quality":         ...
  }},
  "overall_score_100": ...
}}
</review_assessment>
```
"""

In [ ]:
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Scoring with LLM"):
    # Skip if all llm fields are already filled
    if all(pd.notna(row.get(field, pd.NA)) for field in llm_fields):
        continue
    
    prompt = template.format(
        title=row['title'],
        abstract=row['abstract'],
        review_text=row['review_text']
    )

    for attempt in range(1):
        try:
            response = chat("llama3", messages=[{'role': 'user', 'content': prompt}], options={"temperature": 0.0, "seed": 42})
            content = response['message']['content']
            match = pattern.search(content)
            if not match:
                raise ValueError("No JSON block found")

            parsed = json.loads(match.group(1))
            for key, val in parsed["criteria"].items():
                df.at[idx, f"llm_{key}"] = val
            df.at[idx, "llm_overall_score_100"] = parsed["overall_score_100"]

            # Save after every successful row
            df.to_csv(input_file, index=False, quoting=csv.QUOTE_ALL)
            break

        except Exception as e:
            print(f"❌ Error at row {idx}, attempt {attempt + 1}: {e}")
            # time.sleep(0.5)
